In [8]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal
import matplotlib.pyplot as plt

In [2]:
def f(x,y):
    return 418.9829*2 - x*np.sin(np.sqrt(abs(x))) - y*np.sin(np.sqrt(abs(y)))

In [50]:
f.__code__.co_argcount

2

In [121]:
class Gen:
    def __init__(self,dominio,precision):
        self.dominio = dominio
        self.precision = precision
        self.len = self.length(dominio,precision)

    def length(self,dominio,precision):
        """Calcula el número de bits en un gen.

        Parameters
        ----------
        dominio : tuple
            Límites (liminf, limsup) de la variable en cuestión.
        precision : int
            Número de cifras decimales luego del punto para la variable en cuestión.

        Returns
        -------
        int
            Número de bits del gen <-> variable.
        """
        # Limites
        rango = dominio[1] - dominio[0]
        cell_count = math.ceil(np.log2(rango))

        # Decimales
        cell_count += math.ceil(np.log2(10**precision - 1))
        return cell_count
    
    def translate(self,x,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) genes. 

        Parameters
        ----------
        x : float or str
            Gen a traducir (float si es un real, str si es binario).
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str or float
            Gen traducido (str si x era un real, float si x era binario).
        '''
        liminf = self.dominio[0]
        liminf = Decimal(str(liminf))

        if mode == 'code':
            ## Real -> Binario
            
            # Cortar decimales por redondeo
            x = round(x,self.precision)
            # Offset - Signo -> rango>=0
            x = Decimal(str(x))
            x += abs(liminf)

            # Eliminar punto
            x_int = int(str(x).replace(".",""))
            return np.binary_repr(x_int, width=self.len)
        
        elif mode == 'decode':
            ## Binario -> Real

            # Convierte a entero base 10
            x = str(int(x,2)).zfill(self.precision)
            print(x)
            # Añadir el punto decimal
            int_part = x[:-self.precision]
            dec_part = x[-self.precision:].zfill(self.precision)
            print(int_part, dec_part)
            x_float = Decimal(int_part + '.' + dec_part)

            # Extraer offset
            x_float -= abs(liminf)
            return float(x_float)

In [126]:
round(-214.874,6)

-214.874

In [122]:
x1 = Gen((-500,500), 6)

In [123]:
bin1 = x1.translate(-214.874, mode='code')
bin1

'000000000001000101100111000110'

In [124]:
x1.translate(bin1)

285126
 285126


-499.714874

In [56]:
class SimpleEvolutionary:
    # Constructor
    def __init__(self,f,dominios,precisiones):
        """
        Parameters
        ----------
        f : function
            Función a optimizar por el algoritmo.
        dominios : tuple, list
            Iterable que contiene tuplas (liminf, limsup) de los dominios de las variables para la función "f".
        precisiones : tuple, list
            Iterable que contiene el número (int) de cifras decimales luego del punto para cada variable.
        """
        self.f = f
        self.dominios = dominios
        self.precisiones = precisiones

        # Lóngitudes de genes y cromosoma
        self.gen_lens = [self.gen_length(dom,prc) for dom,prc in zip(dominios,precisiones)]
        self.chrom_len = sum(self.gen_lens)
    
    ########################
    #### REPRESENTACIÓN ####
    def gen_length(self,dominio,precision):
        """Calcula el número de bits en un gen.

        Parameters
        ----------
        dominio : tuple
            Límites (liminf, limsup) de la variable en cuestión.
        precision : int
            Número de cifras decimales luego del punto para la variable en cuestión.

        Returns
        -------
        int
            Número de bits del gen <-> variable.
        """
        # Limites
        rango = dominio[1] - dominio[0]
        cell_count = math.ceil(np.log2(rango))

        # Decimales
        cell_count += math.ceil(np.log2(10**precision - 1))
        return cell_count
    
    def gen_translate(self,x,locus,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) cromosomas. 

        Parameters
        ----------
        x : float or str
            Gen a traducir (float si es un real, str si es binario).
        locus : int
            Número de gen en el cromosoma al que pertenece.
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str or float
            Gen traducido (str si x era un real, float si x era binario).
        '''

        if mode == 'code':
            ## Real -> Binario

            x = Decimal(str(x))
            liminf = self.dominios[locus][0]
            liminf = Decimal(str(liminf))
            # Signo -> trasladar a rango>=0
            x += abs(liminf)
            # Eliminar punto
            x_int = str(x).replace(".","")
            return np.binary_repr(x_int, width=self.chrom_len)
        
        elif mode == 'decode':
            ## Binario -> Real

            # Convierte a entero base 10
            x = str(int(x,2))
            # Añadir el punto decimal
            int_part = x[:-self.precision[locus]]
            dec_part = x[self.precision[locus]:]
            return float(int_part + '.' + dec_part)

In [102]:
x = 486.1345678
str(Decimal(str(x)))

'486.1345678'

In [103]:
x = -486.1345678
x = Decimal(str(x))
print(x)
liminf = -500
liminf = Decimal(str(liminf))
print(liminf)
# Signo -> trasladar a rango>=0
x += abs(liminf)
print(x)
# Eliminar punto
x_int = int(str(x).replace(".",""))
print(x_int)
np.binary_repr(x_int)

-486.1345678
-500
13.8654322
138654322


'1000010000111011001001110010'

In [92]:
int(np.binary_repr(x_int),2)

98613478

In [87]:

presicion = 3

x = str(int(x,2))


# Añadir el punto decimal
int_part = x[:-self.precision[locus]]
dec_part = x[self.precision[locus]:]
return float(int_part + '.' + dec_part)

'abcd'

In [70]:
".".join(('312','009'))

'312.009'

In [94]:
## Real -> Binario
x = -421.849878
x = Decimal(str(x))
print(x)
liminf = -500
liminf = Decimal(str(liminf))
print(liminf)
# Signo -> trasladar a rango>=0
x += abs(liminf)
print(x)
# Eliminar punto
x_int = str(x).replace(".","")
print(x_int)
# np.binary_repr(x_int, width=)

-421.849878
-500
78.150122
78150122


### Representación: Transformando reales



In [3]:
def chrom_length2(liminf,limsup,precision):
    '''
    liminf: Dominio inf
    limsup: Dominio sup
    precision: Lugares decimales a conservar
    '''
    # Signo
    cell_count = 0
    if liminf<0 or limsup<0:
        cell_count += 1
    
    # Entero
    max_limits = max(abs(liminf), abs(limsup))
    max_binary = np.binary_repr(int(max_limits))
    cell_count += len(max_binary)

    # Decimal
    cell_count += precision

    return cell_count

In [ ]:
def chrom_structure(x,liminf):
    '''
    x: Elemento a traducir
    liminf: Dominio inf
    '''
    ### Real -> Binario
    x = Decimal(str(x))
    liminf = Decimal(str(liminf))
    # Signo -> trasladar a rango>=0
    x += abs(liminf)
    
    # Eliminar punto
    x_int = str(x).replace(".","")

    return cell_count

In [ ]:
rango = 
math.ceil(np.log2(rango))

In [ ]:
def representacion(x,mode='code'):
    '''
    x: Elemento a traducir
    mode: Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.
    '''
    chromosome_length = 1
    if mode == 'code':
        np.binary_repr(x, width=chromosome_length)
    else:
        #

### Población: Generación aleatoria

In [5]:
def aptitud(binary):
    '''
    Traduce las entradas binarias a entradas reales para ser evaluadas en la función objetivo "f".
    '''
